## Multi Agent-Systems


 A team is a group of agents that work together to achieve a common goal

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)
# model_client_2 = 

# Single Agent Approach

A single agent is going to create a short story for us

In [2]:
from autogen_agentchat.agents import (AssistantAgent)

story_agent = AssistantAgent(
    name = 'story_agent',
    model_client=model_client,
    system_message='You are a creating writer. Generate a short story about a brave knight and a dragon.'
)

In [3]:
from autogen_agentchat.messages import TextMessage

async def test_simple_agent():
    task = TextMessage(
        content='Write a short story about a brave knight and a dragon. Keep it up to 50 words',
        source='user'
    )
    result = await story_agent.run(task=task)
    print(result.messages[-1].content)

await test_simple_agent()

Sir Galen faced the roaring dragon, scales shimmering under the moonlit sky. Heart pounding, he raised his sword, its blade gleaming with courage. As the dragon lunged, Galen saw unexpected fear in its eyes. Lowering his weapon, he offered peace. Together, they forged an unlikely alliance, forever altering their fates.


In a land of whispers, Sir Elric faced the feared dragon Zephyr atop Mistpeak. Rather than battle, they spoke, discovering shared loneliness. Zephyr guarded the kingdom thereafter, tales of violence dissolved into ones of friendship, where fire and steel forged a bond stronger than any blade.

# Multi Agent Team Approach

We will be using 3 agents to now write the story, each with its own work and mastery

In [4]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)


RoundRobinGroupChat is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants= [plot_agent, character_agent, ending_agent],
    max_turns=6
)
team.component_type

'team'

In [6]:

async def test_team():
    task = TextMessage(
        content='Write a short story a brave boy and his Pokemon. Keep it up to 50 words',
        source='user'
    )

    result = await team.run(task=task)
    for each_agent_message in result.messages:
        print(f'{each_agent_message.source} : {each_agent_message.content}')
        

await test_team()

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words
plot_writer : Young Leo and his courageous Charmander ventured through Whispering Woods, seeking the legendary Celestial Gem. Facing wild Beedrill, Charmander’s fiery tail blazed a path to safety. Finally, under the Moonlight Falls, the Celestial Gem glimmered. As they claimed their prize, a new chapter in their adventure beckoned.
character_writer : **Pokemon Character: Blaze the Charmander**

Blaze is a spirited and determined Charmander with a knack for adventure. Its fiery tail mirrors its vibrant personality—always flickering with an eagerness to explore the unknown. Blaze is deeply loyal to Leo, its trainer, and shares an unbreakable bond with him. Raised in the volcanic terrains of Pyra Ridge, Blaze was the runt of its hatchling siblings but soon became the bravest. Motivated by the desire to prove itself, Blaze aims to evolve into a powerful Charizard, conquering formidable challenges along the way. 

user : Write a short story a brave boy and his Pokemon. Keep it up to 50 words



1. plot_writer : In Lumina Town, young Kai and his fiery Charmander ventured into the Whispering Woods. The pair faced the ominous Shadow Grove, where flickering shadows danced. Together, they unleashed a dazzling Flamethrower, dispelling darkness. United by courage and friendship, they emerged victorious, ready for endless adventures ahead.



2. character_writer : In the vibrant world of Lumina Town, Kai's adventurous spirit was matched only by his fiery companion, Charmander. 

**Charmander**: A tiny lizard Pokémon with a flame at the tip of its tail, Charmander is spirited and fiercely loyal. This Charmander bears a unique streak of blue in its flame, symbolizing its unusual potential and heritage. Having grown up in the bustling markets of Lumina Town, it's accustomed to cheer and challenges alike. Charming but determined, Charmander’s motivation is driven by a thirst for growth. It yearns to evolve, not just physically but in strength and wisdom, to protect those it cares about.

Navigating through the lively world, their latest trial lay in the daunting Whispering Woods. At the heart of the woods lurked the villainous Master Umbra.

**Master Umbra**: A shadowy figure cloaked in constant twilight, Umbra was once known as Ardor, a revered Pokémon researcher committed to studying light’s impact on Pokémon evolution. Betrayed and ridiculed by peers for his radical theories, he turned to darkness, embracing the shadows. Master Umbra commands a legion of shadow Pokémon, thriving in dimness he manipulates. His ultimate goal: harnessing darkness to eclipse every region, showing the world power lies not in light, but in the void it resists. Umbra's cunning is overshadowed only by his understanding of light and shadow, but deep down, he desires recognition and validation.

Face-to-face with Master Umbra in the intimidating Shadow Grove, Kai and Charmander find themselves tested. Shadows swirl, threatening to overwhelm, but fueled by bravery and the symbolic blue flame, Charmander releases a scorching Flamethrower. Light pierces darkness, driving back Umbra’s forces.

Unfazed, Umbra vanishes with a vow to reclaim dominion, leaving Kai and Charmander that much closer to their dream: to stand as protectors and champions of balance, ensuring darkness never eclipses the light. 



3. ending_writer : As Kai and Charmander celebrated their victory over Master Umbra, the burnt shadows began to swirl and reform, revealing a hidden message in the grove's ashes. It read: "Look deeper, young one." Intrigued, Kai followed the cryptic hint, discovering an ancient stone tablet with the truth that Umbra, once known as Ardor, was his estranged grandfather. The twist of fate was a reminder that sometimes, the shadows we face are closer than they appear, casting new light on his path ahead.

# Comparision Table

## Single-Agent vs. Multi-Agent Comparison

| Aspect             | Single Agent                | Multi-Agent Team              |
|--------------------|-----------------------------|-------------------------------|
| **Creativity**     | Limited to one viewpoint    | Diverse ideas from each agent |
| **Depth**          | Basic story elements        | Detailed plot, characters, ending |
| **Flexibility**    | Stuck to one style          | Adaptable with specialized roles |
| **Setup Effort**   | Simple, one agent           | More agents, team setup       |

In [7]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=108) metadata={} content="Golden leaves drift in the breeze,  \nUnder a sky of soft, misty greys.  \nWhispering secrets through rustling trees,  \nAs autumn weaves her colorful maze.  \n\nThe crisp air carries tales untold,  \nOf harvest moons and fires aglow.  \nNature's quilt of red and gold,  \nWraps the earth in a cozy show.  \n\nPumpkin scents and apple pies,  \nDance with cinnamon in the air.  \nUnderneath the amber skies,  \nFall's enchantment lingers everywhere.  \n" type='TextMessage'
source='character_writer' models_usage=RequestUsage(prompt_tokens=156, completion_tokens=489) metadata={} content="**Pokemon: Lumispark**  \n\nLumispark is an Electric/Fairy-type Pokémon, with a lithe, ferret-like body coursing with arcs of vibrant electricity. Its fur glows with a soft luminescence th

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat

team2 = RoundRobinGroupChat(
    participants= [plot_agent],
    max_turns=2
)

In [9]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team2.reset()  # Reset the team for a new task.
async for message in team2.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=112) metadata={} content="Leaves descend in golden dance,  \nWhispering winds in nature's trance.  \nCrisp air kisses the rolling hills,  \nWhile amber sunsets give gentle chills.  \n\nPumpkins line the crooked path,  \nHarvest moons in night's cool bath.  \nRusset tones weave forest's cloak,  \nOwls hoot tales as shadows invoke.  \n\nIn fall's embrace, the world transforms,  \nA time of endings, yet life re-forms.  \nNature's rest, a serene retreat,  \nUntil spring's song, when hearts will beat.  " type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=150, completion_tokens=233) metadata={} content="In the spirit of fall, a Pokémon named Emberleaf, a vibrant Leafon, embarks on a journey through the ever-changing landscape of autumn.  \n\nAs leaves of gold and crims

In [10]:
message

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='plot_writer', models_usage=RequestUsage(prompt_tokens=34, completion_tokens=112), metadata={}, content="Leaves descend in golden dance,  \nWhispering winds in nature's trance.  \nCrisp air kisses the rolling hills,  \nWhile amber sunsets give gentle chills.  \n\nPumpkins line the crooked path,  \nHarvest moons in night's cool bath.  \nRusset tones weave forest's cloak,  \nOwls hoot tales as shadows invoke.  \n\nIn fall's embrace, the world transforms,  \nA time of endings, yet life re-forms.  \nNature's rest, a serene retreat,  \nUntil spring's song, when hearts will beat.  ", type='TextMessage'), TextMessage(source='plot_writer', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=233), metadata={}, content="In the spirit of fall, a Pokémon named Emberleaf, a vibrant Leafon, embarks on a journey through

In [11]:
from autogen_agentchat.teams import RoundRobinGroupChat

team3 = RoundRobinGroupChat(
    participants= [plot_agent],
    # max_turns=2
)

In [ ]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team3.reset()  # Reset the team for a new task.
async for message in team3.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=141) metadata={} content='Crimson leaves in gentle flight,  \nWhisper secrets to the night.  \nGolden hues and amber light,  \nIn fall’s embrace, the world ignites.  \n\nCrisp air carries stories old,  \nOf harvest, hearth, and tales retold.  \nUnderneath a harvest moon,  \nLeaves twirl to a rustling tune.  \n\nPumpkin patches, scarecrows stand,  \nGuardians of this vibrant land.  \nNature dons its bold attire,  \nAs autumn sets the heart afire.  \n\nIn every leaf, a journey starts,  \nA tapestry of nature’s arts.  \nFall, with beauty to enthrall,  \nHolds us gently in its thrall.  ' type='TextMessage'
source='plot_writer' models_usage=RequestUsage(prompt_tokens=179, completion_tokens=560) metadata={} content="Sorry, I accidentally provided a poem instead of focusing on a Pokémon's journey. 